##### **Importação de bibliotecas**

In [1]:
import numpy as np
import timeit

In [ ]:
from math import sqrt
import numpy as np
import seaborn as sns
import pandas as pd

##### **Matrizes de teste**

In [3]:
MatrizALU = np.array([[2,1,1,0], [4,3,3,1], [8,7,9,5], [6,7,9,8]])
MatrizBLU= np.array([[2], [3], [1], [-4]])


##### **Códigos das atividades passsadas**

In [4]:
# Matriz Inversa
def retirar_linha_coluna(array, linha_remover, coluna_remover):
    nova_matriz = []

    for i, linha in enumerate(array):
        if i != linha_remover:
            nova_linha = []
            for j, elemento in enumerate(linha):
                if j != coluna_remover:
                    nova_linha.append(elemento)
            nova_matriz.append(nova_linha)
    return nova_matriz

def determinante(array):
    tamanho = len(array)
    if len(array) == 1:
        return array[0][0]
    if len(array) == 2:
        return array[0][0]*array[1][1] - array[0][1]*array[1][0]
    det = 0
    for coluna in range(tamanho):
        submatriz = retirar_linha_coluna(array, 0, coluna)
        cofator = ((-1) ** coluna) * array[0][coluna] * determinante(submatriz)
        det += cofator  
    return det

def cofator(array, i, j):
    submatriz = retirar_linha_coluna(array, i, j)
    cofator_val = ((-1) ** (i + j)) * determinante(submatriz)
    return cofator_val

def matriz_transposta(array):
    transposta = []
    num_linhas = len(array)
    num_colunas = len(array[0])

    for j in range(num_colunas):
        nova_linha = []
        for i in range(num_linhas):
            nova_linha.append(array[i][j])
        transposta.append(nova_linha)
    return transposta


# Eliminação Gaussiana
def resolucao_matriz_triangular_superior(matrizA, matrizB):
    tamanho = len(matrizA)
    solucao = np.zeros(tamanho)
    
    for linha in range(tamanho - 1, -1, -1):
        soma = 0
        for coluna in range(linha + 1, tamanho):
            soma += matrizA[linha][coluna] * solucao[coluna] 
        
        solucao[linha] = (matrizB[linha] - soma) / matrizA[linha][linha] 
        
    return solucao

def eliminacao_gaussiana(MatrizA, MatrizB):
    tamanho = len(MatrizA)
        
    for i in range(tamanho):
        
        for j in range(i + 1, tamanho):
            mij = MatrizA[j][i] / MatrizA[i][i] 
            MatrizB[j] = MatrizB[j] - mij * MatrizB[i] 
            for k in range(i, tamanho):
                MatrizA[j][k] = MatrizA[j][k] - mij * MatrizA[i][k] 
                
    resolucao = resolucao_matriz_triangular_superior(MatrizA, MatrizB)
    return resolucao


##### **Código Decomposição LU**

In [5]:
def decomposicao_LU(MatrizA):
    
    tamanho = len(MatrizA) # Tamanho da matriz
    
    MatrizL = np.eye(tamanho) # Criação de uma matriz identidade para ser a base da MatrizL 
    MatrizU = MatrizA.copy() # Criação de uma matriz base para a MatrizU
    
    # Pivoteamento
    MatrizP = np.eye(tamanho) # Matriz para a permutação
    
    for i in range(tamanho): # Define qual será a linha com o maior valor absoluto para ser o pivô
        linha_max = i
        for k in range(i + 1, tamanho):
            if abs(MatrizU[k, i]) > abs(MatrizU[linha_max, i]):
                linha_max = k
        
    if i != linha_max: # Se a linha em análise não é a linha com o maior valor absoluto na coluna (linha_max), realiza a troca
        MatrizU[[i, linha_max], :] = MatrizU[[linha_max, i], :] # Realiza a troca das linhas
        MatrizP[[i, linha_max], :] = MatrizP[[linha_max, i], :]
        # Os dois pontos são utilizados para garantir que todas as colunas serão englobadas
            
        if i > 0: # Se a linha atual não for a primeira (não toca nela...), também troca os elementos correspondentes na matriz MatrizL
            MatrizL[[i, linha_max], :i] = MatrizL[[linha_max, i], :i]
            
    # Multiplicadores e criação das matrizes L e U
                
    for i in range(tamanho): # Cálculo dos multiplicadores da MatrizA e atualizações das matrizes L e U
        for j in range(i + 1, tamanho):
            mij = MatrizU[j][i] / MatrizU[i][i] 
            MatrizL[j][i] = mij
            for k in range(i, tamanho):
                MatrizU[j, k] -= mij * MatrizU[i, k] 
    return MatrizP, MatrizL, MatrizU

MatrizP, MatrizL, MatrizU = decomposicao_LU(MatrizALU)

def resolucao_matriz_triangular_inferior(matrizA, matrizB): 
    # Função para a resolução de uma matriz triangular inferior baseada na função de resolução para a matriz triangular superior
    tamanho = len(matrizA)
    solucao = np.zeros(tamanho)
    
    for linha in range(tamanho): # A iteração acontece de forma crescente
        soma = 0
        for coluna in range(linha):
            soma += matrizA[linha][coluna] * solucao[coluna] 
        
        solucao[linha] = (matrizB[linha] - soma) / matrizA[linha][linha] 
        
    return solucao

def resolucao_decomposicao_LU(MatrizL, MatrizU, MatrizB, MatrizP):
    MatrizB_perm = MatrizP @ MatrizB  # Permutar o vetor B
    MatrizV = resolucao_matriz_triangular_inferior(MatrizL, MatrizB_perm) # Resolução da Matriz v (sistema Lv = b, sendo v = Uu)
    Matriz_Resposta = resolucao_matriz_triangular_superior(MatrizU, MatrizV) # Resolução da Matriz Resposta (sistema Uu = v )
    return Matriz_Resposta

In [6]:
decomposicao_LU(MatrizALU)
resolucao_decomposicao_LU(MatrizL, MatrizU, MatrizBLU, MatrizP)

C:\Users\julia24002\AppData\Local\Temp\ipykernel_27292\2735442049.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  solucao[linha] = (matrizB[linha] - soma) / matrizA[linha][linha]


array([ 1.,  1., -1., -1.])

##### **Qual é a vantagem do método LU?**

In [7]:
# Teste 1
MatrizA1 = np.array([[2,1,1,0], [4,3,3,1], [8,7,9,5], [6,7,9,8]])
MatrizB1= np.array([[2], [3], [1], [-4]])


# Gaussiana
tempo_inicial = timeit.default_timer()
solucao_gaussiana = eliminacao_gaussiana(MatrizA1, MatrizB1)
tempo_final = timeit.default_timer()
diferenca_de_tempo1 = tempo_final-tempo_inicial
print(f"O tempo utilizado foi de {diferenca_de_tempo1}")

MatrizA3 = np.array([[2,1,1,0], [4,3,3,1], [8,7,9,5], [6,7,9,8]])
MatrizB3= np.array([[3], [4], [5], [10]])

tempo_inicial = timeit.default_timer()
solucao_gaussiana = eliminacao_gaussiana(MatrizA1, MatrizB1)
tempo_final = timeit.default_timer()
diferenca_de_tempo3 = tempo_final-tempo_inicial
print(f"O tempo utilizado foi de {diferenca_de_tempo3}")

print(diferenca_de_tempo1+diferenca_de_tempo3)

O tempo utilizado foi de 0.001354500069282949
O tempo utilizado foi de 0.00039190007373690605
0.001746400143019855


C:\Users\julia24002\AppData\Local\Temp\ipykernel_27292\840781635.py:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  solucao[linha] = (matrizB[linha] - soma) / matrizA[linha][linha]


In [8]:
MatrizA2 = np.array([[2,1,1,0], [4,3,3,1], [8,7,9,5], [6,7,9,8]])
MatrizB2= np.array([[2], [3], [1], [-4]])

# LU
tempo_inicial = timeit.default_timer()
MatrizP, MatrizL, MatrizU = decomposicao_LU(MatrizA2)
solucao_lu = resolucao_decomposicao_LU(MatrizL, MatrizU, MatrizB2, MatrizP)
tempo_final = timeit.default_timer()
diferenca_de_tempo2 = tempo_final-tempo_inicial
print(f"O tempo utilizado foi de {diferenca_de_tempo2}")

MatrizB4= np.array([[3], [4], [5], [10]])
                   
tempo_inicial = timeit.default_timer()
solucao_lu = resolucao_decomposicao_LU(MatrizL, MatrizU, MatrizB4, MatrizP)
tempo_final = timeit.default_timer()
diferenca_de_tempo4 = tempo_final-tempo_inicial
print(f"O tempo utilizado foi de {diferenca_de_tempo4}")

print(diferenca_de_tempo2+diferenca_de_tempo4)

O tempo utilizado foi de 0.0009014999959617853
O tempo utilizado foi de 0.00010499998461455107
0.0010064999805763364


C:\Users\julia24002\AppData\Local\Temp\ipykernel_27292\2735442049.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  solucao[linha] = (matrizB[linha] - soma) / matrizA[linha][linha]


O método LU é especialmente vantajoso para a resolução de mais de um sistema linear, nos quais há apenas diferenças na matriz B. 
Para um sistema linear único, como demonstrado pela diferença nos tempos obtidos para o código acima*, a eliminação gaussiana é mais efetiva. Apesar dos dois métodos apresentarem complexidade O(n^3), isto é, que cresce cubicamente em relação ao número de linhas da matriz, a eliminação gaussiana realiza menos operações, visto que apenas calcula a uma decomposição. No entanto, quando analisamos mais de um sistema linear, considerando a reutilização das decomposições, o método LU apresenta um tempo menor de resolução.

Isso ocorre pois na eliminação gaussiana, é necessário recalcular a decomposição da matriz A sempre que a matriz B é alterada, visto que a transformação em triangular superior também implica na transformação da matriz de resultado. Já na decomposição LU, como a decomposição leva apenas em consideração a matriz A, é necessário apenas recalcular a resolução. 


*A diferença de tempo é mais perceptível em matrizes maiores, em razão da relação entre o aumento da complexidade e uma maior quantidade de linhas

In [11]:
matriza = np.array([[1, 0.1, 0.001, 0.001], [1, 0.2, 0.04, 0.008], [1, 0.3, 0.09, 0.027], [1, 0.4, 0.16, 0.064]])
matrizb = np.array([5, 13, -4, -8])
matrizp, matrizl, matrizu = decomposicao_LU(matriza)
resolucao_decomposicao_LU(matrizl, matrizu, matrizb, matrizp)

array([   453.42857143,  -4475.47619048,  14428.57142857, -15309.52380952])